In [2]:
import pandas as pd
import numpy as np

### Table of Contents
* death data
    - [deceased counts by province as of 2020-03-30](#covid)
* air quality data
    - [2006 ~ 2011 air pollution](#suiAir)
* confounding factors
    - [population as of May 2018](#pop)
    - [elderly population](#elderly)
    - [socioeconomic data](#socio)

<a id='covid'></a>
#### deceased counts by province as of 2020-03-30

In [4]:
covidProv = pd.read_csv('data/raw/Kaggle/TimeProvince.csv').iloc[-17:].drop('time', axis = 1)
covidProv

,date,province,confirmed,released,deceased
1190,2020-03-30,Seoul,426,92,0
1191,2020-03-30,Busan,118,87,3
1192,2020-03-30,Daegu,6624,3837,111
1193,2020-03-30,Incheon,58,15,0
1194,2020-03-30,Gwangju,20,13,0
1195,2020-03-30,Daejeon,34,17,0
1196,2020-03-30,Ulsan,39,19,0
1197,2020-03-30,Sejong,46,12,0
1198,2020-03-30,Gyeonggi-do,463,160,5
1199,2020-03-30,Gangwon-do,36,21,1


In [5]:
covidProv.to_csv('data/cleaned/covid.csv', index = False)

<a id='suiAir'></a>
#### 2006 ~ 2011 air pollution

In [16]:
suiAir = pd.read_excel('data/raw/Air_Pollution_Suicide_Paper/air-pollution-suiside-data.XLSX', 
                       index_col=[0, 1], skiprows = 1)

# related columns
col = []
for i in np.arange(len(suiAir.columns)):
    if all(word not in suiAir.columns[i] for word in ['suicide', 'average', 'celebrity', '_lag']):
        col = np.append(col, suiAir.columns[i])
suiAir = suiAir[col].iloc[:, :-3]

# check that there's no missing dates
# print(sum(~(suiAir.index.get_level_values(1).sort_values() == suiAir.index.get_level_values(1))))
# sum(~(suiAir.index.get_level_values(1) == pd.date_range(start = suiAir.index.get_level_values(1)[0], end = suiAir.index.get_level_values(1)[-1], freq = 'W')))

# weekly aggregation
suiAirM = pd.DataFrame(suiAir.mean(axis = 0)).reset_index()

# clean air pollution species & region code 
spRegCode = suiAirM['index'].values
l = len(spRegCode)
species = []
regCode = []
for i in np.arange(l):
    if any(word in spRegCode[i] for word in ['temp', 'suns']):
        species = np.append(species, spRegCode[i][:4])
        regCode = np.append(regCode, int(spRegCode[i][4:]))
    else:
        species = np.append(species, spRegCode[i][:-2])
        regCode = np.append(regCode, spRegCode[i][-2:])
suiAirM = pd.concat([suiAirM, pd.Series(species), pd.Series(pd.to_numeric(regCode))], axis = 1)
suiAirM.columns = ['spRegCode', 'mean', 'species', 'regCode']

# wide format
wSuiAir = suiAirM.pivot(index='regCode', columns='species', values='mean')

# add region names
# Kyunggi is Gyeonggi-do
# Chungbuk is an abbreviation of Chungcheongbuk-do
# Chungnam is Chungcheongnam-do
# Jeonbuk is Jeollabuk-do
# Jeonnam is Jeollanam-do
# Gyungbuk is Gyeongsangbuk-do
# Gyungnam is Gyeongsangnam-do
r = ' 1 (Seoul), 2 (Busan), 3 (Daegu), 4 (Incheon), 5 (Gwangju), 6 (Daejeon), 7 (Ulsan), 8 (Gyeonggi-do), 9 (Gangwon-do), 10 (Chungcheongbuk-do), 11 (Chungcheongnam-do), 12 (Jeollabuk-do), 13 (Jeollanam-do), 14 (Gyeongsangbuk-do), 15 (Gyeongsangnam-do), 16 (Jeju-do)'
rSep = [x.split(' ')[1:] for x in r.split(',')]
rCode = [int(x[0]) for x in rSep]
rName = [x[1][1:-1] for x in rSep]
regNameCode = pd.DataFrame({'regCode': rCode, 'regName': rName})
wSuiAir = pd.merge(wSuiAir, regNameCode, on='regCode')

# select and rename relevant columns
suiAQ = wSuiAir[['regName', 'co', 'dust', 'no2', 'ozon', 'so2']] # dust is pm10
suiAQ.columns = ['province', 'co', 'pm10', 'no2', 'ozone', 'so2']
suiAQ

,province,co,pm10,no2,ozone,so2
0,Seoul,0.592560,54.363761,0.035602,0.018958,0.005527
1,Busan,0.412848,52.025483,0.021389,0.025593,0.005644
2,Daegu,0.574088,51.182769,0.024137,0.022440,0.005521
3,Incheon,0.597216,59.858056,0.029855,0.022116,0.007306
4,Gwangju,0.555272,48.558421,0.021364,0.023613,0.003809
5,Daejeon,0.593732,45.822265,0.020796,0.020791,0.004376
6,Ulsan,0.461698,50.758603,0.022792,0.022761,0.007844
7,Gyeonggi-do,0.622958,61.335920,0.030399,0.020228,0.005871
8,Gangwon-do,0.661480,53.770866,0.018038,0.025448,0.005049
9,Chungcheongbuk-do,0.690050,59.620265,0.021432,0.022694,0.006002


In [18]:
suiAQ.to_csv('data/cleaned/suiAQ.csv', index = False)

<a id='pop'></a>
#### population as of May 2018

In [12]:
pop = pd.read_csv('data/raw/SKPopulation/SKPopulation.csv', thousands = ',')
pop = pop.dropna()
pop['province'] = ['Seoul','Busan','Daegu','Incheon','Gwangju','Daejeon','Ulsan','Sejong','Gyeonggi-do','Gangwon-do','Chungcheongbuk-do','Chungcheongnam-do',
'Jeollabuk-do','Jeollanam-do','Gyeongsangbuk-do','Gyeongsangnam-do','Jeju-do']
pop = pop[['province', 'Population']]
pop

,province,Population
1,Seoul,9830452.0
3,Busan,3460707.0
4,Daegu,2471136.0
5,Incheon,2952476.0
6,Gwangju,1460972.0
7,Daejeon,1496123.0
8,Ulsan,1161303.0
10,Sejong,295041.0
12,Gyeonggi-do,12941604.0
13,Gangwon-do,1545452.0


In [14]:
pop.to_csv('data/cleaned/SKpopulation.csv', index = False)

<a id='elderly'></a>
#### elderly population

In [19]:
old = pd.read_excel('data/raw/Elderly/statistic_id1085308_elderly-population-south-korea-2019-by-region.xlsx', sheet_name = 'Data', skiprows = 4, thousands = ',')
old = old.dropna(axis = 1)
old['province'] = ['Gyeonggi-do','Seoul','Busan','Gyeongsangbuk-do','Gyeongsangnam-do','Jeollanam-do','Chungcheongnam-do','Incheon','Daegu','Jeollabuk-do','Gangwon-do','Chungcheongbuk-do','Daejeon','Gwangju','Ulsan','Jeju-do','Sejong']
old = old.iloc[:, [1, 2]]
old.columns = ['elderly', 'province']
old['elderly'] = old['elderly'] * 1000
old

,elderly,province
0,1590000,Gyeonggi-do
1,1402000,Seoul
2,591000,Busan
3,527000,Gyeongsangbuk-do
4,524000,Gyeongsangnam-do
5,396000,Jeollanam-do
6,372000,Chungcheongnam-do
7,370000,Incheon
8,368000,Daegu
9,355000,Jeollabuk-do


In [20]:
old.to_csv('data/cleaned/elderly.csv', index = False)

<a id='socio'></a>
#### socioeconomic data

In [21]:
socio = pd.read_csv('data/raw/Kaggle/Region.csv')
socio = socio.loc[:, ['province', 'elementary_school_count', 'kindergarten_count', 'university_count', 'nursing_home_count']]
socio = socio.groupby('province').sum()
socio

,elementary_school_count,kindergarten_count,university_count,nursing_home_count
province,,,,
Busan,608,816,44,13504
Chungcheongbuk-do,518,656,34,5538
Chungcheongnam-do,818,998,42,7282
Daegu,458,710,22,10166
Daejeon,296,520,30,5968
Gangwon-do,698,736,36,5038
Gwangju,310,624,34,5704
Gyeonggi-do,2554,4474,122,40982
Gyeongsangbuk-do,942,1414,66,8948


In [22]:
socio.to_csv('data/cleaned/socio.csv')